In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import torch
import torch.nn.functional as F

dataset = pd.read_csv('/Users/elvis/ITProjects/GitHub/jupyterPY/try_vscode/raw_price_train/2_r_price_train.csv')

FileNotFoundError: [Errno 2] File b'/Users/elvis/ITProjects/GitHub/jupyterPY/try_vscode/raw_price_train/2_r_price_train.csv' does not exist: b'/Users/elvis/ITProjects/GitHub/jupyterPY/try_vscode/raw_price_train/2_r_price_train.csv'

## 数据预处理

In [0]:
# from sklearn.preprocessing import OneHotEncoder
# onehotencoder = OneHotEncoder(categorical_features = [0])
def preprocess(data,m):
    '''
    data: the dataframe of stock price
    m: the length of sequence
    '''
    volume = data["Volume"].tolist()
    adj_close = data["Adj Close"].tolist()
    #
    res_X = []
    res_y = []
    # 剔除前4个元素
    for i in range(4,len(adj_close)-m):
        res_X.append(adj_close[i:i+m])
        res_y.append(adj_close[i+m])
    return res_X,res_y

## 建立浅神经网络model

In [0]:
import torch
class model_nn(torch.nn.Module):
    # Net类的初始化函数
    def __init__(self, n_feature, n_hidden, n_output):
        # 继承父类的初始化函数
        super(model_nn, self).__init__()
        # 网络的隐藏层创建，名称可以随便起
        self.hidden1_layer = torch.nn.Linear(n_feature, n_hidden)
#         self.hidden2_layer = torch.nn.Linear(n_hidden, n_hidden)
        # 输出层(预测层)创建，接收来自隐含层的数据
        self.predict_layer = torch.nn.Linear(n_hidden, n_output)

     # 网络的前向传播函数，构造计算图
    def forward(self, x):
        # 用relu函数处理隐含层输出的结果并传给输出层
#         hidden1_result = self.hidden1_layer(x)
#         relu_hidden1_result = F.relu(hidden1_result)
        
#         hidden2_result = self.hidden2_layer(relu_hidden1_result)
#         relu_result = F.relu(hidden2_result)
        hidden1_result = self.hidden1_layer(x)
        relu_result = F.relu(hidden1_result)
        
        predict_result = self.predict_layer(relu_result)
        return predict_result

In [0]:
net = model_nn()

In [0]:
# 训练次数
TRAIN_TIMES = 10000
# 输入输出的数据维度
INPUT_FEATURE_DIM = 5
OUTPUT_FEATURE_DIM = 1
# 隐含层中神经元的个数
NEURON_NUM = 11
# 学习率，越大学的越快，但也容易造成不稳定，准确率上下波动的情况
LEARNING_RATE = 0.1

In [0]:
def run_nn():
    ## 建立x，y
    x_data,y_data = preprocess(dataset,INPUT_FEATURE_DIM)
    x_data = torch.from_numpy(np.array(x_data))
    y_data = torch.from_numpy(np.array(y_data))
    print(x_data.size())
    print(y_data.size())
    # 建立网络
    net = model_nn(n_feature=INPUT_FEATURE_DIM, n_hidden=NEURON_NUM, n_output=OUTPUT_FEATURE_DIM)
    # print(net)
    # 训练网络
    # 这里也可以使用其它的优化方法
    optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)
    # 定义一个误差计算方法
    loss_func = torch.nn.MSELoss()
    for i in range(TRAIN_TIMES):
        # 输入数据进行预测
        prediction = net(x_data.float())
        # 计算预测值与真值误差，注意参数顺序问题
        # 第一个参数为预测值，第二个为真值
        loss = loss_func(prediction, torch.unsqueeze(y_data, dim=1).float())
        if(i == TRAIN_TIMES - 1):
            print("NEURON_NUM: %d" %NEURON_NUM)
            print(loss)
        # 开始优化步骤
        # 每次开始优化前将梯度置为0
        optimizer.zero_grad()
        # 误差反向传播
        loss.backward()
        # 按照最小loss优化参数
        optimizer.step()


In [0]:
run_nn()